# Exploring Other Preset Agents in AutoGen AgentChat

## Overview

While `AssistantAgent` is a versatile general-purpose agent, AutoGen AgentChat provides several other specialized preset agents designed for specific interaction patterns and tasks. Understanding these agents can help you choose the right tool for your multi-agent applications.

This notebook will briefly introduce some of these other preset agents and their primary use cases.

## Prerequisites

Ensure you have the necessary packages installed:

In [1]:
! pip install --quiet -U "autogen-agentchat>=0.7" "autogen-ext[openai]>=0.7" rich

Note: Some agents require additional dependencies, which are included in the `autogen-ext` extras.

## `UserProxyAgent`

The `UserProxyAgent` is designed to simulate a human user. It takes user input and returns it as responses, often acting as the entry point for a conversation or a proxy for human feedback and intervention in multi-agent systems.

We haven't covered teams - this will be covered in a later tutorial.  For now just think of a team as multiple agents that work together until some termination criteria.

In [7]:
import asyncio
from autogen_agentchat.agents import UserProxyAgent, AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.ui import Console

async def run_user_proxy_example():
    model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
    
    # The assistant agent
    assistant = AssistantAgent(
        name="assistant",
        model_client=model_client,
        system_message="You are a helpful AI assistant."
    )
    
    # The user proxy agent
    user_proxy = UserProxyAgent(
        name="user_proxy",
    )
    
    print("\n🔄 Conversation with UserProxyAgent:")
    print("Enter TERMINATE to exit.")
    print("-" * 40)
    
    from autogen_agentchat.teams import RoundRobinGroupChat
    from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
    
    # Create a team with termination condition
    team = RoundRobinGroupChat(
        participants=[assistant, user_proxy],
        termination_condition=TextMentionTermination("TERMINATE") | MaxMessageTermination(max_messages=5)
    )
    
    # Run the team conversation
    await Console(team.run_stream(task="Tell me a joke."))    
    await model_client.close()

await run_user_proxy_example()


🔄 Conversation with UserProxyAgent:
----------------------------------------
---------- TextMessage (user) ----------
Tell me a joke.
---------- TextMessage (assistant) ----------
Why did the scarecrow win an award?

Because he was outstanding in his field!


Enter your response:  what do you call a donkey with three legs


---------- TextMessage (user_proxy) ----------
what do you call a donkey with three legs
---------- TextMessage (assistant) ----------
A wonky donkey!


Enter your response:  TERMINATE


---------- TextMessage (user_proxy) ----------
TERMINATE


## `CodeExecutorAgent`

The `CodeExecutorAgent` is specialized in executing code. It can be configured to run code in various environments (e.g., a local Python interpreter, a Docker container). This agent is crucial for tasks requiring computation, data processing, or interaction with local systems.

In [18]:
import asyncio
from autogen_agentchat.agents import CodeExecutorAgent, ApprovalRequest, ApprovalResponse
from autogen_agentchat.messages import TextMessage
from autogen_ext.code_executors.local import LocalCommandLineCodeExecutor
from autogen_core import CancellationToken

# This function asks the user for permission to run the code.
def ask_for_approval(request: ApprovalRequest) -> ApprovalResponse:
    """A simple function to get user approval for code execution."""
    print("Approval requested to run the following code:")
    print("---------------------------------------------")
    print(request.code)
    print("---------------------------------------------")
    
    reply = input("Execute code? (y/n): ").strip().lower()
    if reply == 'y':
        # The fix is here: we now provide a reason for the approval.
        return ApprovalResponse(approved=True, reason="User approved.")
    return ApprovalResponse(approved=False, reason="User denied.")

async def main() -> None:
    # 1. Create an agent that executes code locally.
    #    It will use the `ask_for_approval` function before running anything.
    local_executor = LocalCommandLineCodeExecutor(work_dir="coding")
    code_agent = CodeExecutorAgent(
        name="code_executor",
        code_executor=local_executor,
        approval_func=ask_for_approval
    )

    # 2. Define the task for the agent.
    code_task = TextMessage(
        content="Here is the code to run: ```python\nprint('Hello from AutoGen!')\n```",
        source="user",
    )

    # 3. Run the agent and print the result.
    response = await code_agent.on_messages([code_task], CancellationToken())
    print("\nAgent's response:")
    print(response.chat_message.content)

await main()

Approval requested to run the following code:
---------------------------------------------
```python
print('Hello from AutoGen!')

```
---------------------------------------------


Execute code? (y/n):  y



Agent's response:
Hello from AutoGen!



In [ ]:
## Other Agents

### `OpenAIAssistantAgent`

The `OpenAIAssistantAgent` is a specialized agent that leverages OpenAI's Assistants API. This agent can utilize tools, retrieve information, and manage threads directly through the OpenAI platform, offering a different paradigm for agent development compared to the local `AssistantAgent`. [[api docs](https://microsoft.github.io/autogen/stable/reference/python/autogen_ext.agents.openai.html#autogen_ext.agents.openai.OpenAIAssistantAgent)]

### `MultimodalWebSurfer`

The `MultimodalWebSurfer` is an agent capable of interacting with web pages. It can navigate, extract information, and even perform actions on websites, making it ideal for tasks like web research, data scraping, or interacting with web-based applications. [[api docs](https://microsoft.github.io/autogen/stable/reference/python/autogen_ext.agents.web_surfer.html#autogen_ext.agents.web_surfer.MultimodalWebSurfer)]

### `FileSurfer`

The `FileSurfer` agent is designed to interact with local file systems. It can search, read, and potentially write to files, making it invaluable for tasks involving local data analysis, code review, or document processing. [[api docs](https://microsoft.github.io/autogen/stable/reference/python/autogen_ext.agents.file_surfer.html#autogen_ext.agents.file_surfer.FileSurfer)]

### `VideoSurfer`

The `VideoSurfer` agent is designed to process and extract information from video content. This can involve summarizing video transcripts, identifying key moments, or answering questions based on video data. [[api docs](https://microsoft.github.io/autogen/stable/reference/python/autogen_ext.agents.video_surfer.html#autogen_ext.agents.video_surfer.VideoSurfer)]

## Next Steps

Explore these preset agents further:

1.  **Explore the api docs**: and attempt to use other agents listed in this tutorial.
3.  **Customization**: Understand how to extend or customize these agents for your specific needs.